In [1]:
from utils import *

In [119]:
with open('pickles/go_dict.pkl', 'r') as fin:
    go_dict = pickle.load(fin)
with open('pickles/protein_dict.pkl', 'r') as fin:
    protein_dict = pickle.load(fin)
with open('pickles/gene_dict.pkl', 'r') as fin:
    gene_dict = pickle.load(fin)

In [8]:
import cPickle as pickle
import pandas as pd

def filter_df_by_seq_len(df, lengths=(20, 700)):
    min_length, max_length = lengths
    return df[(df['sequence'].apply(len) >= min_length) & (df['sequence'].apply(len) <= max_length)]

def filter_df_by_sim():
    pass

In [121]:
columns = protein_dict.itervalues().next().keys()
columns_data = [[] for c in columns]
allergen_set = set()
for k,v in protein_dict.iteritems():
    if v['allergen'] == True and v['sequence'] not in allergen_set:
        allergen_set.add(v['sequence'])
        for i,c in enumerate(columns):
            columns_data[i].append(v[c])
uniprot_allergen_df = pd.DataFrame(dict(tuple(zip(columns, columns_data))))

columns = protein_dict.itervalues().next().keys()
columns_data = [[] for c in columns]
non_allergen_set = set()
for k,v in protein_dict.iteritems():
    if v['sequence'] not in allergen_set and v['sequence'] not in non_allergen_set:
        non_allergen_set.add(v['sequence'])
        for i,c in enumerate(columns):
            columns_data[i].append(v[c])
uniprot_non_allergen_df = pd.DataFrame(dict(tuple(zip(columns, columns_data))))

In [123]:
uniprot_allergen_df.head(10)

,acc_num,allergen,db,ec_num,entry_name,gene_names,organism_name,other_acc_nums,protein_name,sequence
0,Q4WCS7,True,Swiss-Prot,1.11.1.15,PMP20_ASPFU,"[pmp20, AFUA_6G02280]",330879,[O43099],Putative peroxiredoxin pmp20,MSGLKAGDSFPSDVVFSYIPWSEDKGEITACGIPINYNASKEWADK...
1,P15494,True,Swiss-Prot,None,BEV1A_BETPN,"[BETVIA, BETVI]",3505,[Q96369],Major pollen allergen Bet v 1-A,MGVFNYETETTSVIPAARLFKAFILDGDNLFPKVAPQAISSVENIE...
2,Q9T0M8,True,Swiss-Prot,None,PROF2_PARJU,[PRO2],33127,[],Profilin-2,MSWQAYVDDHLMCDVGDGNTLASAAIIGHDGSVWAQSANFPQLKPE...
3,Q39967,True,Swiss-Prot,None,ALL5_HEVBR,[],3981,[],Major latex allergen Hev b 5,MASVEVESAATALPKNETPEVTKAEETKTEEPAAPPASEQETADAT...
4,P45431,True,Swiss-Prot,None,BEV1B_BETPN,[BETV1B],3505,[],Major pollen allergen Bet v 1-B,MGVFNYETETTSVIPAARLFKAFILEGDTLIPKVAPQAISSVENIE...
5,Q43483,True,Swiss-Prot,None,IAA1_HORVU,[IAM1],4513,[P16968],Alpha-amylase inhibitor BMAI-1,PTSVAVDQGSMVSNSPGEWCWPGMGYPVYPFPRCRALVKSQCAGGQ...
6,A4GDU5,True,Swiss-Prot,None,PROBM_OLEEU,[],4146,[],Profilin-2,MSWQTYVDDHLMCDIEGHEGHRLTAAAIVGQDGSVWAQSATFPQFK...
7,A4GDU3,True,Swiss-Prot,None,PROBL_OLEEU,[],4146,[],Profilin-4,MSWQSYVDDHLMCDIEGHEGHRLTAAAIVGHDGSVWAQSATFPQFK...
8,A4GDU2,True,Swiss-Prot,None,PROBK_OLEEU,[],4146,[],Profilin-3,MSWQTYVDDHLMCDIEGHEGHRLTAAAIVGHDGSVWAQSATFPQFK...
9,A4GDU0,True,Swiss-Prot,None,PROBJ_OLEEU,[],4146,[],Profilin-2,MSWQTYVDDHLMCDIEGHEGHRLTAAAIVGHDGSVWAQSATFPQFK...


In [6]:
uniprot_non_allergen_df.head(1)

,acc_num,allergen,db,ec_num,entry_name,gene_names,organism_name,other_acc_nums,protein_name,sequence
0,C6DJ78,False,Swiss-Prot,None,TTCA_PECCP,"[ttcA, PC1_2316]",561230,[],tRNA 2-thiocytidine biosynthesis protein TtcA,MSENQQINQKQQYNLNKLQKRLRRNVGEAIADFNMIEEGDRIMVCL...


In [7]:
print uniprot_allergen_df.shape, uniprot_non_allergen_df.shape

(601, 10) (464738, 10)


In [32]:
fasta_columns = ['db', 'identifier', 'entry_name', 'protein_name', 'organism_name', 'gene_name', 'sequence']
parsed_fasta = parse_fasta_str(open('data/allergen_online_allergens.fasta').read())
ao_df = pd.DataFrame(data=parsed_fasta, columns=fasta_columns)

ao_identifiers = ao_df['identifier'].tolist()
for i in range(0, len(ao_identifiers), 200):
    if i == 0:
        ao_df = convert_acc_nums_to_df(str(ao_identifiers[i:i+200]))
    else:
        ao_df = ao_df.append(convert_acc_nums_to_df(str(ao_identifiers[i:i+200])))

In [42]:
ao_df[ao_df['db'] == 'tr'].head(1)

,db,identifier,entry_name,protein_name,organism_name,gene_name,sequence
0,tr,B0KZJ6,B0KZJ6_ACASI,Allergen Aca s 13,Acarus siro,None,MVQINGSYKLEKSDNFDAFLKELGLNFVTRNLAKSATPTVEVSVNG...


In [43]:
#includes entries from the tremble database as well as swiss-prot
ao_set = set(ao_df['sequence'])
print len(ao_set)
print len(ao_set.difference(allergen_set))
print len(ao_set.difference(non_allergen_set))
print len(ao_set.difference(allergen_set, non_allergen_set))

1611
1163
1437
989


In [44]:
final_allergen_sequences = [s for s in allergen_set.union(ao_set.difference(non_allergen_set))] 

allergen_df = pd.DataFrame({'sequence': final_allergen_sequences})
allergen_df.shape

non_allergen_sequences = [s for s in non_allergen_set.difference(ao_set)]
non_allergen_df = pd.DataFrame({'sequence': non_allergen_sequences})
non_allergen_df.shape

# with open('pickles/allergen_df.pkl', 'w') as fout:
#     pickle.dump(allergen_df, fout)
    
# with open('pickles/non_allergen_df.pkl', 'w') as fout:
#     pickle.dump(non_allergen_df, fout)

In [2]:
# Uncomment this code to run this cell without running whole notebook
with open('pickles/non_allergen_df.pkl', 'r') as fin:
    non_allergen_df = pickle.load(fin)
with open('pickles/allergen_df.pkl', 'r') as fin:
    allergen_df = pickle.load(fin)

allergen_df['allergen'] = 1
non_allergen_df['allergen'] = 0
allergen_df = filter_df_by_seq_len(allergen_df)
non_allergen_df = filter_df_by_seq_len(non_allergen_df)

# allergen_df = add_protein_characteristics(allergen_df)
# non_allergen_df = add_protein_characteristics(non_allergen_df)

# with open('pickles/allergen_df_with_features.pkl', 'w') as fout:
#     pickle.dump(allergen_df, fout)
    
# with open('pickles/non_allergen_df_with_features.pkl', 'w') as fout:
#     pickle.dump(non_allergen_df, fout)

In [39]:
# def add_protein_characteristics(df):
#     df = df.copy()
#     aa_list = ['A', 'C','E','D','G','F','I','H','K','M','L','N','Q','P','S','R','T','W','V','Y']
#     aa_dict = {}
#     for aa in aa_list:
#         aa_dict[aa] = []
#     prop_dict = {'aromaticity': [], 'instability_index': [], 'helix': [], 'turn': [], 'sheet': [],
#                 'isoelectric_point': [], 'gravy': []} #, 'flexibility': []}
#     for i, s in enumerate(df['sequence']):
#         s = s.replace('B', 'D').replace('Z', 'E').replace('J', 'L').replace('X', 'G').replace('U', 'C').replace('O', 'K')
#         pa = ProteinAnalysis(s)
#         prop_dict['aromaticity'].append(pa.aromaticity())
#         prop_dict['instability_index'].append(pa.instability_index())
#         prop_dict['isoelectric_point'].append(pa.isoelectric_point())
#         prop_dict['gravy'].append(pa.gravy())
#         #prop_dict['flexibility'].append(pa.flexibility())
#         for fraction, ss in zip(pa.secondary_structure_fraction(), ['helix', 'turn', 'sheet']):
#             prop_dict[ss].append(fraction)
#         for k, v in pa.get_amino_acids_percent().items():
#             aa_dict[k].append(v)
#     for k, v in aa_dict.items():
#         df[k] = v
#     for k, v in prop_dict.items():
#         df[k] = v
#     return df

# %timeit -n 1 -r 1 add_protein_characteristics(non_allergen_df.ix[0:10000])

1 loop, best of 1: 14.3 s per loop


# AllerHunter

In [9]:
base = 'data/allerHunterData/'
paths = ['testingdata/testing.allergen.fa', 'testingdata/testing.homolog.fa', 'testingdata/testing.putative_non_allergen.fa',
        'trainingdata/training.allergen.fa', 'trainingdata/training.homolog.fa', 'trainingdata/training.putative_non_allergen.fa',
        'independentdata/indp.allergen.fa', 'independentdata/indp.homolog.fa', 'independentdata/indp.putative_non_allergen.fa']
seqs = []

for path in paths:
    with open(base + path) as fin:
        seqs.append(get_sequences_from_fasta(fin))

ah_test_allergens, ah_test_APNs, ah_test_DPNs = seqs[0:3]
ah_train_allergens, ah_train_APNs, ah_train_DPNs = seqs[3:6]
ah_ind_allergens, ah_ind_APNs, ah_ind_DPNs = seqs[6:9]

ah_allergen_df = pd.DataFrame({'sequence': ah_test_allergens + ah_train_allergens})
ah_non_allergen_df = pd.DataFrame({'sequence': ah_test_APNs + ah_test_DPNs + ah_train_APNs + ah_train_DPNs})
ah_ind_allergen_df = pd.DataFrame({'sequence': ah_ind_allergens})
ah_ind_non_allergen_df = pd.DataFrame({'sequence': ah_ind_APNs + ah_ind_DPNs})

ah_allergen_df['allergen'] = 1
ah_non_allergen_df['allergen'] = 0
ah_ind_allergen_df['allergen'] = 1
ah_ind_non_allergen_df['allergen'] = 0

base = 'pickles/'
paths = ['ah_allergen_df_with_features.pkl', 'ah_non_allergen_df_with_features.pkl', 'ah_ind_allergen_df_with_features.pkl',
        'ah_ind_non_allergen_df_with_features.pkl']
dataframes = [ah_allergen_df, ah_non_allergen_df, ah_ind_allergen_df, ah_ind_non_allergen_df]
for p,d in zip(paths, dataframes):
    d = add_protein_characteristics(d)
    d = filter_df_by_seq_len(d)
    with open(base + p, 'w') as fout:
        pickle.dump(d, fout)

# Playground

In [126]:
fasta_columns = ['db', 'identifier', 'entry_name', 'protein_name', 'organism_name', 'gene_name', 'sequence']
parsed_fasta = parse_fasta_str(open('data/allergen_online_allergens.fasta').read())
ao_df = pd.DataFrame(data=parsed_fasta, columns=fasta_columns)

In [135]:
np.unique(ao_df['protein_name'].apply(lambda x: x[0:-2])).shape

(850,)

In [131]:
ao_df

,db,identifier,entry_name,protein_name,organism_name,gene_name,sequence
0,tr,B0KZJ6,B0KZJ6_ACASI,Allergen Aca s 13,Acarus siro,None,MVQINGSYKLEKSDNFDAFLKELGLNFVTRNLAKSATPTVEVSVNG...
1,tr,B0KZK1,B0KZK1_ACASI,Alpha-amylase,Acarus siro,None,MAHLLLAVVAITLALSQSVFGGSPYSNPHFTGSRSVITHLMQWKFD...
2,tr,L7TV16,L7TV16_9ERIC,Kiwellin,Actinidia arguta,Kwl1,MAQLTLLLLSLFLTLISLPPPGASISSCNGPCRDLNDCDGQLICIK...
3,tr,L7TT88,L7TT88_9ERIC,Kiwellin,Actinidia arguta,Kwl1,MAQLTLLLLSLFFTLISLPPPGASISSCNGPCRDLNDCNGQLICIK...
4,tr,L7TY99,L7TY99_9ERIC,Kiwellin,Actinidia arguta,Kwl1,MAQLTLLLLSLFLTLISLPPPGASISSCNGPCRDLNDCDGQLICIK...
5,sp,P00785,ACTN_ACTCH,Actinidain,Actinidia chinensis,None,MGLPKSFVSMSLLFFSTLLILSLAFNAKNLTQRTNDEVKAMYESWL...
6,sp,P85204,NLTP1_ACTCH,Non-specific lipid-transfer protein 1 (Fragment),Actinidia chinensis,None,AVSCGQVDTALTPCL
7,tr,L7TT83,L7TT83_ACTCH,Kiwellin,Actinidia chinensis,Kwl1,MAQLSLLLLSLFLTLISLPPPGASISSCNGPCRDLNDCDGQLICIK...
8,tr,D1YSM4,D1YSM4_ACTCH,Bet v 1 related allergen,Actinidia chinensis,ypr-10,MGVVTYDMEIPSKVPPVKLYKAFILDGDTLVPKVLPHAIKCVKILE...
9,sp,P83958,TLP_ACTCH,Thaumatin-like protein (Fragment),Actinidia chinensis,tlp,ATFNFINNCPFTVWAAAVPG


In [152]:
v = 'heye'
v.replace('e', 'u')

'huyu'

In [153]:
v

'heye'